<a href="https://colab.research.google.com/github/ioradlin/NAT_project/blob/main/NAT_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
def to_txt(x, name):
  with open(name, 'w') as file:
      file.write(x)

def read_txt(name):
  with open(name, 'r') as file:
      temp= file.read()
  return temp

## Transformer code

based on : https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

In [170]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [171]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            mask = mask.to(attn_scores.device) #@@ 해야되는거였음???
            #print(attn_scores.device) #cuda:0
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [172]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [173]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [174]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [175]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [176]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):

      # src mask
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
      # tgt mask
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)

        return output

In [177]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, 30))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, 40))  # (batch_size, seq_length)

In [178]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(1):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.663016319274902




---



## NAT Encoder

In [179]:
import torch
import torch.nn as nn

## Layer는 transformer와 완전히 동일

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

# --------------------------------


class NATEncoder(nn.Module):
    def __init__(self, src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(NATEncoder, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src):

        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        return src_mask

    def forward(self, src):
        src_mask = self.generate_mask(src)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)


        return enc_output

In [180]:
#Encoder (fertility predictor)
src_vocab_size = 1000; d_model = 512; num_heads = 8; num_layers = 6
d_ff = 2048; max_seq_length = 100; dropout = 0.1

encoder = NATEncoder(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [181]:
#Example

src_sequence = torch.randint(0, src_vocab_size, (1, 10))
out= encoder(src_sequence)

out.shape

torch.Size([1, 10, 512])

## ✨ NAT Decoder

In [182]:
## 수정 중!

## Self -> Cross(Position) -> Cross(EnC_out) -> fc 순

class NAT_DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(NAT_DecoderLayer, self).__init__()

        #for NAT PE attention
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        # used for both NAT, transformer
        self.self_attn = MultiHeadAttention(d_model, num_heads) #forward: Q,K,V,mask
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.norm4 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask) #query의 자기 위치를 가리는 mask
        x = self.norm1(x + self.dropout(attn_output))   # (어차피 x와 더하니까 주변 정보를 더 반영하려는 의도일듯)

        # PE가 Q, K고 decoder state가 V / PE는 어차피 고정 값이니까 그냥 만들면 됨
        pe_x= self.positional_encoding(x) #docoder state의 shape만 알면 됨
        attn_output= self.cross_attn(pe_x, pe_x, x) # mask는 .. 안 주면 될듯 ? (mask: QK로 만든 seq x seq 크기의 score에 적용)
        x = self.norm2(x + self.dropout(attn_output))
        #print('pe', pe_x.shape) # batch x tgt_seq x d_model

        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm3(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm4(x + self.dropout(ff_output))
        return x

In [183]:
## 수정 중 (빠른 디버깅을 위해 먼저 해봐도됨, training은 NAT로 함)

import torch
import torch.nn as nn

class NAT_Decoder(nn.Module):
    def __init__(self, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(NAT_Decoder, self).__init__()
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.decoder_layers = nn.ModuleList([NAT_DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, tgt):
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return tgt_mask

    def forward(self, tgt, enc_output):
        tgt_mask = self.generate_mask(tgt)
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, None, tgt_mask)  # Removed src_mask

        output = self.fc(dec_output)

        return output


In [184]:
# Create a random input sequence for the decoder (just for demonstration)
tgt_sequence = torch.randint(0, tgt_vocab_size, (1, 8))  # Assuming output sequence length is 8

# Instantiate the TransformerDecoder
transformer_decoder = NAT_Decoder(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

enc_output = torch.randn(1, 10, d_model)
output = transformer_decoder(tgt_sequence, enc_output)

print("Output size:", output.size())

Output size: torch.Size([1, 8, 5000])




---



## Data Preprocess

#### Load datasets

#### Load corpus, vocab, fertility label

In [185]:
import pickle

with open('train_ft_bpe.pkl', 'rb') as file:
     ft = pickle.load(file)

with open('src_vocab.pkl', 'rb') as file:
     src_vocab = pickle.load(file)

with open('tgt_vocab.pkl','rb') as file:
     tgt_vocab = pickle.load(file)

In [186]:
## BPE tokenized

bpe_de= read_txt('bpe_de.txt')
bpe_en= read_txt('bpe_en.txt')

#### **dataset** preprocess

In [187]:
de= bpe_de.split('\n')
en= bpe_en.split('\n')

In [188]:
from collections import Counter
lengths= [len(d) for d in de]

i = 0
for length in lengths:
  if length<=512:
    i+=1

i/len(lengths)

## 99% 이상이 512 이상 길이이므로, 512를 max_len으로 사용

0.9959310779523087


id로 전환, padding & truncation으로 max_len 통일

In [189]:
def tokens_to_ids(text, vocab, max_seq_len=512):
    preprocessed = []

    for line in text:
        tokens = line.split()
        ids = [vocab.get(token, vocab.get('[UNK]')) for token in tokens]

        preprocessed.append(ids)

    return preprocessed

In [190]:
tokened_de= tokens_to_ids(de, src_vocab)
tokened_en= tokens_to_ids(en, tgt_vocab)

####  ⚠️ fertility issue ⚠️

fertility 이상한거 해결 해야함, 지금은 일단넘어감

In [191]:
#shape check (안맞으면 training 안됨)

i=0
j=0
for d, f, e in zip(tokened_de, ft, tokened_en):

  if len(d)!=len(f):
    i+=1
  if sum(f)!=len(e):
    j+=1
    #print(sum(f),len(e))


print(i,j) #일단 버렸음

1985 77512


In [192]:
data=[]

for d, e, f in zip(tokened_de, tokened_en, ft):
  if len(d)==len(f) and sum(f)==len(e):
    tgt = []
    for token_to_repeat,num_repeat in zip(d,f):
      tgt.extend([token_to_repeat]*num_repeat)

    data.append({'de':d,'en':e ,'tgt': tgt, 'ft':f})

In [193]:
len(data)

81904

In [194]:
data[2]

{'de': [43, 93, 75, 87, 94, 95, 96, 97, 10, 98, 73, 13],
 'en': [2, 44, 89, 90, 52, 85, 59, 91, 35, 6, 92, 55, 93, 94, 15],
 'tgt': [43, 43, 43, 93, 87, 87, 94, 95, 95, 96, 97, 97, 10, 73, 13],
 'ft': [3, 1, 0, 2, 1, 2, 1, 2, 1, 0, 1, 1]}

#### ✨ Loader & collate_fn

In [195]:
# PAD token이 원래 0인데, CE 계산할때 ignore = 0 해주면 fertility 예측값인 0도 무시되서
# PAD token의 인덱스를 새로운 값 10135로 바꿔야할거같음 (tgt_vocab은 이보다 짧음)

pad_id= len(src_vocab)

In [196]:
import torch
from torch.nn.utils.rnn import pad_sequence

def tensor_pad_collate_fn(batch, pad_id=pad_id, max_sequence_length=512):
    padded_data = {}

    max_lens = {key: min(max(len(item[key]) for item in batch if key in item), max_sequence_length) for key in batch[0].keys()}

    for key in batch[0].keys():
        if key in max_lens:
            max_len = max_lens[key]
            sequences = [torch.tensor(item[key][:max_len] if len(item[key]) > max_len else item[key] + [pad_id] * (max_len - len(item[key])), dtype=torch.long) for item in batch if key in item]
            padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=pad_id)
            padded_data[key] = padded_sequences

    return padded_data


##✨ NAT

위 Encoder, Decoder을 굳이 쓰지 않고 Layer만 가져와서 만들면 됨

* NAT 에서는 target sentence가 forward에 사용되지 않음

In [205]:
class NAT(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(NAT, self).__init__()

        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(src_vocab_size, d_model) ## NAT니까 !?
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([NAT_DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size) #ENC -> DEC -> output
        self.fertility = nn.Linear(d_model, 50) #for ENC -> fertility, L=50 form paper

        self.dropout = nn.Dropout(dropout)


    def generate_mask(self, src, tgt):

        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

        tgt_length = tgt.size(1)
        diag_mask = ~torch.eye(tgt_length, tgt_length).bool()

        # 각각 torch.Size([1, 1, 1, src.size(1)]) torch.Size([tgt_length, tgt_length])
        return src_mask, diag_mask


    def forward(self, src, tgt, ft=None):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)

        return output

In [206]:
src_vocab_size = len(src_vocab)+1 # pad를 len(src_vocab)으로 지정했음!! (fertility 예측값인 0과 혼동 방지용)
tgt_vocab_size = len(tgt_vocab)
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 512
dropout = 0.1

nat = NAT(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
device = torch.device("cuda")
#device = torch.device("cpu") ## @@ 디버깅용
nat.to(device)

print(device)

cuda




---



## ✨ NAT Training

In [208]:
from torch.utils.data import DataLoader

batch_size = 32
criterion = nn.CrossEntropyLoss(ignore_index=10135)
optimizer = optim.Adam(nat.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

#### wandb

In [214]:
%%capture
!pip install wandb -qU

In [215]:
import wandb
wandb.login()

wandb.init(project='nat-first-training')
wandb.watch(nat, criterion, log="all", log_freq=100) #log_freq가 log 남기는 주기

#10 batch면 3200데이터마다 loss 찍음 = 에폭당 50번 정도..?! 말이 안되는뎅
#GPT는 10 step이라고 함. = 320데이터마다 loss = 에폭당 500번.

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: iorad (abcd_team). Use `wandb login --relogin` to force relogin


[]

Training loop

In [217]:
#1 Epoch에 6분 46초, loss 4.3 (data 버린것 수정하면 12분 정도 예상)
#wandb 써서 느려진건지? 16분에 1 에폭 돌았다. 낼 다시 돌려보죠
#일단 초반 loss 변화는 무난한거같당

nat.train()

for epoch in range(10):

    for i, batch in enumerate(train_loader):

      src_data = batch['de'].to(device)
      copied_src = batch['tgt'].to(device)
      tgt_data = batch['en'].to(device)
      #print(src_data.shape, copied_src.shape, tgt_data.shape)

      optimizer.zero_grad()
      output = nat(src_data, copied_src)

      loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data.contiguous().view(-1))
      wandb.log({"CE loss": loss.item()})
      loss.backward()
      optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

In [ ]:
#필요하면 저장
#torch.save(nat.state_dict(), 'nat_epoch10')



---



## Encoder fc (fertility) Training (따로 진행)

다른 부분 다 학습 안되게 freeze 해두고 진행 (training code도 손봐야할듯)

In [218]:
# criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # pad_id: special token index / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
# optimizer = optim.Adam(encoder.parameters(), lr=0.01, betas=(0.9, 0.98), eps=1e-9)
# train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

# encoder.train()

# for epoch in range(20):

#     for i, batch in enumerate(train_loader):

#         src = batch['de'].to(device)
#         ft_label = batch['ft'].to(device)

#         optimizer.zero_grad()

#         output = encoder(src)

#         # L= 50 (fertility 범위: 0~49) from paper
#         loss = criterion(output.contiguous().view(-1,50), ft_label.contiguous().view(-1))

#         loss.backward()

#         optimizer.step()


#     print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

In [219]:
#torch.save(encoder.state_dict(), 'NATencoder_epoch20')

lr 낮추니까 CE loss는 확 줄어드는데, 여전히 1 Epoch 안되서 빠르게 수렴해버림

(한 Epoch을 다 돌지 않았을 때는 Epoch내 데이터의 loss는 validation loss나 마찬가진데도)

### Example (참고용, transformer)

In [219]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 5
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (16, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (16, max_seq_length))  # (batch_size, seq_length)

In [219]:
criterion = nn.CrossEntropyLoss(ignore_index=0) #0: special token / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data)

    loss = criterion(output.contiguous().view(-1,tgt_vocab_size), tgt_data.contiguous().view(-1))

    # print(output.contiguous().view(-1,tgt_vocab_size).shape, tgt_data.contiguous().view(-1).shape)
    # print('loss',loss)
    loss.backward()
    optimizer.step()
    # break
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")